# NS Homeshopping Feasibility Test

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_1_5_pro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fgetting-started%2Fintro_gemini_1_5_pro.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/getting-started/intro_gemini_1_5_pro.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_1_5_pro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


## Step1) Video Work - Get Image
#### Home Shopping video extraction

In [63]:
from google.cloud import storage
from google.cloud import bigquery
import pandas as pd
import time
# Google Cloud Storage 클라이언트 초기화
project_id = "jc-gcp-project"
bucket_name = "jc-gcp-project-01"
destination_blob_name = "pdf_test/capture_images"
storage_client = storage.Client(project=project_id) 

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)

In [61]:
# import cv2
# from google.cloud import storage

# def detect_and_capture_changes(video_path, threshold=400000, bucket_name="jc-gcp-project-01", blob_prefix="pdf_test/capture_images"):
#     """
#     동영상에서 화면 변화를 감지하고 Google Cloud Storage에 캡처 이미지를 저장하는 함수

#     Args:
#         video_path (str): 동영상 파일 경로
#         threshold (int): 변화 감지 임계값 (기본값: 50000)
#         bucket_name (str): Google Cloud Storage 버킷 이름
#         blob_prefix (str): 저장할 blob의 prefix
#     """

#     cap = cv2.VideoCapture(video_path)
#     prev_frame = None
#     frame_count = 0

#     # Google Cloud Storage 클라이언트 초기화
#     project_id = "jc-gcp-project"
#     bucket_name = "jc-gcp-project-01"
#     destination_blob_name = "pdf_test/capture_images"
#     storage_client = storage.Client(project=project_id) 

#     bucket = storage_client.bucket(bucket_name)
#     blob = bucket.blob(destination_blob_name)
    
    
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break

#         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#         gray = cv2.GaussianBlur(gray, (21, 21), 0)

#         if prev_frame is None:
#             prev_frame = gray
#             continue

#         frame_delta = cv2.absdiff(prev_frame, gray)
#         thresh = cv2.threshold(frame_delta, 25, 255, cv2.THRESH_BINARY)[1]
#         thresh = cv2.dilate(thresh, None, iterations=2)
#         cnts, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#         for c in cnts:
#             if cv2.contourArea(c) < threshold:
#                 continue

#             # 변화 감지 시 프레임 캡처 및 Google Cloud Storage에 저장
#             filename = f"captured_image/frame_{frame_count}.jpg"
#             cv2.imwrite(filename, frame)
#             blob = bucket.blob(f"{blob_prefix}{filename}")
#             blob.upload_from_filename(filename)
#             frame_count += 1

#         prev_frame = gray

#     cap.release()

In [87]:
import cv2
import time
from google.cloud import storage
import os

def detect_and_capture_changes(video_path, threshold=50000, bucket_name="jc-gcp-project-01", blob_prefix="pdf_test/capture_images", upload_interval=1):
    """
    동영상에서 화면 변화를 감지하고 Google Cloud Storage에 캡처 이미지를 저장하는 함수

    Args:
        video_path (str): 동영상 파일 경로
        threshold (int): 변화 감지 임계값 (기본값: 50000)
        bucket_name (str): Google Cloud Storage 버킷 이름
        blob_prefix (str): 저장할 blob의 prefix
        upload_interval (int): 초 단위 업로드 간격
    """

    cap = cv2.VideoCapture(video_path)
    prev_frame = None
    frame_count = 0
    last_upload_time = 0

    # Google Cloud Storage 클라이언트 초기화
    project_id = "jc-gcp-project"
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.bucket(bucket_name)
    
    if not os.path.exists("captured_image"):
         os.makedirs("captured_image")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)

        if prev_frame is None:
            prev_frame = gray
            continue

        frame_delta = cv2.absdiff(prev_frame, gray)
        thresh = cv2.threshold(frame_delta, 25, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.dilate(thresh, None, iterations=2)
        cnts, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for c in cnts:
            if cv2.contourArea(c) < threshold:
                continue

            # 현재 프레임의 시간 정보 가져오기
            frame_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000  # 초 단위
            timestamp = time.strftime("%H_%M_%S", time.gmtime(frame_time))  # 시간 정보 포맷팅

            current_time = time.time()
            if current_time - last_upload_time >= upload_interval:
                # 변화 감지 시 프레임 캡처 및 Google Cloud Storage에 저장
                filename = f"captured_image/frame_{timestamp}.jpg"
                cv2.imwrite(filename, frame)
                blob_name = f"{blob_prefix}/frame_{timestamp}.jpg"
                blob = bucket.blob(blob_name)

                try:
                    blob.upload_from_filename(filename)
                    print(f"Uploaded: {blob_name}")
                except Exception as e:
                     print(f"Error uploading {blob_name}: {e}")
                finally:
                     os.remove(filename)
                frame_count += 1
                last_upload_time = current_time
        prev_frame = gray
    cap.release()

In [96]:
start_time = time.time()
detect_and_capture_changes('ns_video_sample.mp4')
end_time = time.time()
elapsed_time = end_time - start_time # 실행시간 계산
print(f"실행 시간:{elapsed_time:.5f} 초")

Uploaded: pdf_test/capture_images/frame_00_00_00.jpg
Uploaded: pdf_test/capture_images/frame_00_00_04.jpg
Uploaded: pdf_test/capture_images/frame_00_00_09.jpg
Uploaded: pdf_test/capture_images/frame_00_00_11.jpg
Uploaded: pdf_test/capture_images/frame_00_00_13.jpg
Uploaded: pdf_test/capture_images/frame_00_00_14.jpg
Uploaded: pdf_test/capture_images/frame_00_00_16.jpg
Uploaded: pdf_test/capture_images/frame_00_00_20.jpg
Uploaded: pdf_test/capture_images/frame_00_00_21.jpg
Uploaded: pdf_test/capture_images/frame_00_00_28.jpg
Uploaded: pdf_test/capture_images/frame_00_00_33.jpg
Uploaded: pdf_test/capture_images/frame_00_00_39.jpg
Uploaded: pdf_test/capture_images/frame_00_00_42.jpg
Uploaded: pdf_test/capture_images/frame_00_00_43.jpg
Uploaded: pdf_test/capture_images/frame_00_00_44.jpg
Uploaded: pdf_test/capture_images/frame_00_00_49.jpg
Uploaded: pdf_test/capture_images/frame_00_00_58.jpg
Uploaded: pdf_test/capture_images/frame_00_00_59.jpg
Uploaded: pdf_test/capture_images/frame_00_01_

Threshold별 이미지 갯수
* 500000 = 24장
* 600000 = 23장
* 1000000 = 5장

## Step2) Text Extraction (from Image)

In [3]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel

PROJECT_ID = "jc-gcp-project"
LOCATION = "us-central1"
#MODEL = "gemini-1.5-pro-002"
# MODEL = "gemini-pro-experimental"
MODEL = "gemini-2.0-flash-exp"

vertexai.init(project=PROJECT_ID, location=LOCATION)
model = GenerativeModel(MODEL)

In [4]:
# @title Helper Function
def generate(prompt:str, img_uri:str, response_schema)->str:
    
    """
      Function to call Gemini for text Extraction.

      Args:
        prompt: Extract all the text on the screen every 10 seconds.
        yt_uri: Youtube video URI to analyze.

      Returns:
        A text extraction with specified response schema.(JSON)

    """

    from vertexai.generative_models import (
            Part,
            SafetySetting,
            GenerationConfig
      )

      # Configuration to contorl LLM generation.
    generation_config = GenerationConfig(
          max_output_tokens = 8192,
          temperature=1,
          top_p =0.95,
          response_mime_type="application/json",
          response_schema=response_schema
      )

      # Sate setting
    safety_settings = [
          SafetySetting(
              category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
              threshold=SafetySetting.HarmBlockThreshold.OFF
          ),
          SafetySetting(
              category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
              threshold=SafetySetting.HarmBlockThreshold.OFF
          ),
          SafetySetting(
              category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
              threshold=SafetySetting.HarmBlockThreshold.OFF
          ),
          SafetySetting(
              category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
              threshold=SafetySetting.HarmBlockThreshold.OFF
          ),
      ]

    image = Part.from_uri(
          mime_type="image/jpeg",
          uri=img_uri,
      )

    responses = model.generate_content(
          [prompt, image],
          generation_config=generation_config,
          safety_settings=safety_settings,
          stream=False,
      )

    return responses.text

In [5]:
img_url = "gs://jc-gcp-project-us-notebooks/us-central1-a/instance-20231027-002630/captured_image/frame_8.jpg"

In [42]:
import json

prompt_transcript = """
You are an expert in image analysis and text extraction. Your task is to analyze a provided image, extract all text from it, and output the extracted text in JSON format according to the user's specifications.

Here's how you should proceed:

1. Analyze the provided image: Carefully examine the image to identify all textual content present within it.

2. Handle multiple numbers: If you encounter multiple numbers appearing together, such as "122424 (124)", separate them into distinct entries, "122424" and "124".

3. JSON formatting: Format the extracted text into a JSON structure as specified by the user. Ensure the JSON structure accurately reflects the extracted text and adheres to the provided format.

4. Handle Korean Text: If the image contains Korean text, ensure it is extracted and represented correctly in the JSON output.

5. Error Handling: If you are unable to extract text from the image, output the following message: "Error: Text extraction from the provided image was not possible. Please provide a clear image or check the image format."
"""

# Response schema can control the output of generation from Gemini.
response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "location": { "type": "STRING",},
            "start_time": { "type": "STRING",},
            "end_time": { "type": "STRING",},
            "elapsed_time": { "type": "STRING",},
            "text_extract": {
                "type": "ARRAY",
                "items" : {
                  "type": "OBJECT",
                  "properties": {
                    "extracted_text": { "type": "STRING",},
                  }
                }
            },
        },
        "required": ["start_time","end_time","elapsed_time"],
    },
}

json_str = generate(prompt_transcript, img_url, response_schema)
json_transcript = json.loads(json_str)

In [43]:
json_transcript

[{'start_time': '00:00:00',
  'end_time': '00:00:05',
  'elapsed_time': '00:00:05',
  'text_extract': [{'extracted_text': '(주)삼성금거래소'},
   {'extracted_text': 'SAMSUNG GOLD EXCHANGE Co., Ltd,'},
   {'extracted_text': '삼성금거래소 골드바 특별전'},
   {'extracted_text': '(주)삼성금거래소'},
   {'extracted_text': 'SAMSUNG GOLD EXCHANGE Go., Ltd.'},
   {'extracted_text': '선택1'},
   {'extracted_text': '실속형 골드바 10g'},
   {'extracted_text': '23년 NS SHOP+'},
   {'extracted_text': 'Shop+'},
   {'extracted_text': '브랜드 일반 정보'},
   {'extracted_text': '164만원'},
   {'extracted_text': '무23'},
   {'extracted_text': '한달 68,333x무24'},
   {'extracted_text': '(주)삼성금거래소/한국'},
   {'extracted_text': '(내부 시착 시 교환/반품 가능,'},
   {'extracted_text': '외부 착용시 불가능)'},
   {'extracted_text': '1등 주얼리 브랜드!'},
   {'extracted_text': '(23,01-12 NS SHOP+ 주얼리브랜드 상품 총 주문액 기준)'},
   {'extracted_text': 'NS홈쇼핑'},
   {'extracted_text': '친구추가'},
   {'extracted_text': '자동 080-677-1234'},
   {'extracted_text': '상담(발신자부담)'},
   {'extracted_text': '1551-

In [44]:
# 결과를 저장할 딕셔너리
result_dict = {}

# 텍스트 추출 항목에 번호를 붙여 딕셔너리에 추가
for i, item in enumerate(json_transcript[0]['text_extract']):
    result_dict[i+1] = item['extracted_text']

print(result_dict)

{1: '(주)삼성금거래소', 2: 'SAMSUNG GOLD EXCHANGE Co., Ltd,', 3: '삼성금거래소 골드바 특별전', 4: '(주)삼성금거래소', 5: 'SAMSUNG GOLD EXCHANGE Go., Ltd.', 6: '선택1', 7: '실속형 골드바 10g', 8: '23년 NS SHOP+', 9: 'Shop+', 10: '브랜드 일반 정보', 11: '164만원', 12: '무23', 13: '한달 68,333x무24', 14: '(주)삼성금거래소/한국', 15: '(내부 시착 시 교환/반품 가능,', 16: '외부 착용시 불가능)', 17: '1등 주얼리 브랜드!', 18: '(23,01-12 NS SHOP+ 주얼리브랜드 상품 총 주문액 기준)', 19: 'NS홈쇼핑', 20: '친구추가', 21: '자동 080-677-1234', 22: '상담(발신자부담)', 23: '1551-1234'}


## Step3) Get PDF Data

In [45]:
# @title Helper Function
def generate_compare(prompt:str, pdf_url:str, response_schema)->str:
    
    from vertexai.generative_models import (
            Part,
            SafetySetting,
            GenerationConfig
      )

      # Configuration to contorl LLM generation.
    generation_config = GenerationConfig(
          max_output_tokens = 8192,
          temperature=1,
          top_p =0.95,
          response_mime_type="application/json",
          response_schema=response_schema
      )

      # Sate setting
    safety_settings = [
          SafetySetting(
              category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
              threshold=SafetySetting.HarmBlockThreshold.OFF
          ),
          SafetySetting(
              category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
              threshold=SafetySetting.HarmBlockThreshold.OFF
          ),
          SafetySetting(
              category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
              threshold=SafetySetting.HarmBlockThreshold.OFF
          ),
          SafetySetting(
              category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
              threshold=SafetySetting.HarmBlockThreshold.OFF
          ),
      ]

    pdf_url = Part.from_uri(
          mime_type="application/pdf",
          uri=pdf_url,
      )  
    
    responses = model.generate_content(
          [prompt, pdf_url],
          generation_config=generation_config,
          safety_settings=safety_settings,
          stream=False,
      )

    return responses.text

In [10]:
pdf_url = "gs://jc-gcp-project-us-notebooks/us-central1-a/instance-20231027-002630/ns_pdf_7.pdf"

In [46]:
import json

dict_str = json.dumps(result_dict, ensure_ascii=False)

# 앞뒤에 문자열 붙이기
prefix = """따라오는 json형태의 text들을 pdf파일의 text들과 비교해주세요.
pdf파일은 정답지 입니다.
json형태로 들어간 프롬프트 내의 text를 기준으로 답하세요.
이미지속 "한달"의 비용은 정답지의 "가격정보 매가(원)"을 무이자 개월수로 나누고 소수 첫째자리에서 반올림한 값을 비교해 주세요.
"무24"와 같이 "무이자개월수"를 의미하는 줄임말도 있습니다. 의미를 파악해서 처리해 주세요.
"""
suffix = """ 
예시:
{"json_transcript": "삼성금거래소 골드바 특별전", "pdf": "없음"}, 
{"json_transcript": "선택1", "pdf": "없음"},
{"json_transcript": "실속형 골드바 10g", "pdf": "(주)삼성금거래소 24K 순금 99.99% 실속형 골드바 10g"}

"""

prompt_transcript = prefix + dict_str + suffix

response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "text_extract": {
                "type": "ARRAY",
                "items" : {
                  "type": "OBJECT",
                  "properties": {
                    "json_transcript": { "type": "STRING",},
                    "pdf": { "type": "STRING",}
                  }
                }
            },
        },
        # "required": ["start_time","end_time","elapsed_time"],
    },
}

# Response schema can control the output of generation from Gemini.
result = generate_compare(prompt_transcript, pdf_url, response_schema)
json_transcript_answer = json.loads(result)

In [47]:
json_transcript_answer

[{'text_extract': [{'json_transcript': '(주)삼성금거래소', 'pdf': '(주)삼성금거래소'},
   {'json_transcript': 'SAMSUNG GOLD EXCHANGE Co., Ltd,', 'pdf': '없음'},
   {'json_transcript': '삼성금거래소 골드바 특별전', 'pdf': '없음'},
   {'json_transcript': '(주)삼성금거래소', 'pdf': '(주)삼성금거래소'},
   {'json_transcript': 'SAMSUNG GOLD EXCHANGE Go., Ltd.', 'pdf': '없음'},
   {'json_transcript': '선택1', 'pdf': '없음'},
   {'json_transcript': '실속형 골드바 10g',
    'pdf': '(주)삼성금거래소 24K 순금 99.99% 실속형 골드바 10g'},
   {'json_transcript': '23년 NS SHOP+', 'pdf': '없음'},
   {'json_transcript': 'Shop+', 'pdf': '없음'},
   {'json_transcript': '브랜드 일반 정보', 'pdf': '없음'},
   {'json_transcript': '164만원', 'pdf': '1640000'},
   {'json_transcript': '무23', 'pdf': '24'},
   {'json_transcript': '한달 68,333x무24',
    'pdf': '매가(원) 1640000 / 24개월 = 68333.33, 반올림 68333, json의 값은 68333 x 24 = 1639992 로 계산과 다름'},
   {'json_transcript': '(주)삼성금거래소/한국', 'pdf': '(주)삼성금거래소'},
   {'json_transcript': '(내부 시착 시 교환/반품 가능,', 'pdf': '없음'},
   {'json_transcript': '외부 착용시 불가능)',

In [48]:
# 텍스트 추출 결과를 저장할 리스트
extracted_data = []

# 텍스트 추출 결과를 리스트에 추가
for item in json_transcript_answer[0]['text_extract']:
    extracted_data.append({'json_transcript': item['json_transcript'], 'pdf': item['pdf']})

# DataFrame으로 변환
df = pd.DataFrame(extracted_data)

In [49]:
df

,json_transcript,pdf
0,(주)삼성금거래소,(주)삼성금거래소
1,"SAMSUNG GOLD EXCHANGE Co., Ltd,",없음
2,삼성금거래소 골드바 특별전,없음
3,(주)삼성금거래소,(주)삼성금거래소
4,"SAMSUNG GOLD EXCHANGE Go., Ltd.",없음
5,선택1,없음
6,실속형 골드바 10g,(주)삼성금거래소 24K 순금 99.99% 실속형 골드바 10g
7,23년 NS SHOP+,없음
8,Shop+,없음
9,브랜드 일반 정보,없음


## Step4) Compare the data

In [50]:
# @title Helper Function
def generate_comp_res(prompt:str, response_schema)->str:
    
    from vertexai.generative_models import (
            Part,
            SafetySetting,
            GenerationConfig
      )

      # Configuration to contorl LLM generation.
    generation_config = GenerationConfig(
          max_output_tokens = 8192,
          temperature=1,
          top_p =0.95,
          response_mime_type="application/json",
          response_schema=response_schema
      )

      # Sate setting
    safety_settings = [
          SafetySetting(
              category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
              threshold=SafetySetting.HarmBlockThreshold.OFF
          ),
          SafetySetting(
              category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
              threshold=SafetySetting.HarmBlockThreshold.OFF
          ),
          SafetySetting(
              category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
              threshold=SafetySetting.HarmBlockThreshold.OFF
          ),
          SafetySetting(
              category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
              threshold=SafetySetting.HarmBlockThreshold.OFF
          ),
      ]

    responses = model.generate_content(
          [prompt, pdf_url],
          generation_config=generation_config,
          safety_settings=safety_settings,
          stream=False,
      )

    return responses.text

In [57]:
column_1 = df.json_transcript
column_2 = df.pdf

prompt_transcript = f"""
You are a meticulous data comparator. Your task is to compare two columns of data provided by the user, row by row, and identify matches, mismatches, and skipped rows based on specific criteria.

**Input Data:**

* Column 1: `{column_1}`
* Column 2: `{column_2}`

**Comparison Logic:**

Follow these steps precisely for each row:

1. **Check for "없음":** If either column in the current row contains "없음", mark the row as "Skipped" and move to the next row.

2. **Numeric Comparison:** Check if both columns contain numbers that appear similar.  "Similar" is defined as numbers having the same magnitude and units (if applicable).  If similar numbers are present, they must match exactly. If they match, mark the row as "Match (Numeric)". If they don't match, mark the row as "Mismatch (Numeric)".

3. **Semantic Comparison:** If no similar numbers are found, compare the meaning of the content in both columns. If the meaning is the same, mark the row as "Match (Semantic)". If the meaning is different, mark the row as "Mismatch (Semantic)".

**Output Format:**

Present your results in a clear, structured format as follows:

```
Row Number | Column 1 Value | Column 2 Value | Status        | Reason
---------------------------------------------------------------------------------
1          | ...            | ...            | Match (Numeric) | Numbers match
2          | ...            | ...            | Skipped       | "없음" present
3          | ...            | ...            | Mismatch (Semantic) | Meaning differs
...
```

**Example:**

Let's say the input columns are:

Column 1:  `["10 apples", "없음", "red car", "5km"]`
Column 2:  `["10 apples", "5 oranges", "blue car", "5 miles"]`

The output should be:

```
Row Number | Column 1 Value | Column 2 Value | Status        | Reason
---------------------------------------------------------------------------------
1          | 10 apples      | 10 apples      | Match (Numeric) | Numbers and meaning match
2          | 없음          | 5 oranges      | Skipped       | "없음" present
3          | red car       | blue car       | Mismatch (Semantic) | Meaning differs (color)
4          | 5km           | 5 miles       | Mismatch (Numeric) | Numbers similar but units differ
```

**Column 1:** {column_1}

**Column 2:** {column_2}
"""

# Response schema can control the output of generation from Gemini.
response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "location": { "type": "STRING",},
            "start_time": { "type": "STRING",},
            "end_time": { "type": "STRING",},
            "elapsed_time": { "type": "STRING",},
            "text_extract": {
                "type": "ARRAY",
                "items" : {
                  "type": "OBJECT",
                  "properties": {
                    "extracted_text": { "type": "STRING",},
                  }
                }
            },
        },
        "required": ["start_time","end_time","elapsed_time"],
    },
}

json_str = generate(prompt_transcript, img_url, response_schema)
json_transcript = json.loads(json_str)

In [58]:
compare_result = []
for i in range(len(json_transcript)):
    # print(json_transcript[i]['text_extract'][0]['extracted_text'])
    compare_result.append({'compare_result': json.loads(json_transcript[i]['text_extract'][0]['extracted_text'])['Status']})
    df_add = pd.DataFrame(compare_result)
    df['comp_res'] = df_add['compare_result']

In [59]:
df

,json_transcript,pdf,comp_res
0,(주)삼성금거래소,(주)삼성금거래소,Match (Semantic)
1,"SAMSUNG GOLD EXCHANGE Co., Ltd,",없음,Skipped
2,삼성금거래소 골드바 특별전,없음,Skipped
3,(주)삼성금거래소,(주)삼성금거래소,Match (Semantic)
4,"SAMSUNG GOLD EXCHANGE Go., Ltd.",없음,Skipped
5,선택1,없음,Skipped
6,실속형 골드바 10g,(주)삼성금거래소 24K 순금 99.99% 실속형 골드바 10g,Mismatch (Semantic)
7,23년 NS SHOP+,없음,Skipped
8,Shop+,없음,Skipped
9,브랜드 일반 정보,없음,Skipped
